<a href="https://colab.research.google.com/github/ElizavetaNosova/CompLing_homeworks/blob/master/disambiguationHW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install Cython numpy
!pip install git+https://github.com/lopuhin/python-adagram.git

  Cloning https://github.com/lopuhin/python-adagram.git to /tmp/pip-req-build-9ipb_3yn
  Running command git clone -q https://github.com/lopuhin/python-adagram.git /tmp/pip-req-build-9ipb_3yn
  Created wheel for adagram: filename=adagram-0.0.1-cp36-cp36m-linux_x86_64.whl size=464631 sha256=11718cce4aaafd5ae5defd12a1b3812491f3881fdf51293f387b128a8c98fa51
  Stored in directory: /tmp/pip-ephem-wheel-cache-bnivvcpa/wheels/11/0f/46/f5df96670df8f7973b4c2311ffc9b02e435a7bd3207f992c4d
Successfully built adagram


In [1]:
!pip install pymorphy2

In [3]:
import adagram
from lxml import html
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
import nltk
nltk.download('stopwords')
from pymorphy2 import MorphAnalyzer
from string import punctuation
import json, os
from collections import Counter
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))



def tokenize(text):
    
    words = [word.strip(punct) for word in text.lower().split() if word and word not in stops]
    words = [word for word in words if word]

    return words

def normalize(text):
    
    words = tokenize(text)
    words = [morph.parse(word)[0].normal_form for word in words if word]

    return words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
from google.colab import files
uploaded = files.upload()

Задание 1

In [0]:
uploaded = files.upload()

In [0]:
import pandas as pd

In [0]:
corpus_xml = html.fromstring(open('paraphrases.xml', 'rb').read())
texts_1 = []
texts_2 = []
classes = []

for p in corpus_xml.xpath('//paraphrase'):
    texts_1.append(p.xpath('./value[@name="text_1"]/text()')[0])
    texts_2.append(p.xpath('./value[@name="text_2"]/text()')[0])
    classes.append(p.xpath('./value[@name="class"]/text()')[0])
    
data = pd.DataFrame({'text_1':texts_1, 'text_2':texts_2, 'label':classes})
data['text_1_norm'] = data['text_1'].apply(normalize)
data['text_2_norm'] = data['text_2'].apply(normalize)

In [0]:
data

,text_1,text_2,label,text_1_norm,text_2_norm
0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,0,"[полицейский, разрешить, стрелять, поражение, ...","[полиция, мочь, разрешить, стрелять, хулиган, ..."
1,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,0,"[право, полицейский, проникновение, жилища, ре...","[правило, внесудебный, проникновение, полицейс..."
2,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,0,"[президент, египет, ввести, чрезвычайный, поло...","[власть, египет, угрожать, ввести, страна, чре..."
3,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,-1,"[вернуться, сирия, россиянин, волновать, вопро...","[самолёт, мчс, вывезти, россиянин, разрушить, ..."
4,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,0,"[москва, сирия, вернуться, 2, самолёт, мчс, ро...","[самолёт, мчс, вывезти, россиянин, разрушить, ..."
...,...,...,...,...,...
7222,Путин освободил от должности ряд генералов,Путин снял с должностей более 20 руководителей...,0,"[путин, освободить, должность, ряд, генералов]","[путин, снять, должность, 20, руководитель-сил..."
7223,Облака над Москвой в День Победы разгонят девя...,Путеводитель по Дню Победы: как провести 9 мая...,-1,"[облако, москва, день, победа, разогнать, девя...","[путеводитель, день, победа, провести, 9, май,..."
7224,Любляна отпразднует День Победы вместе с Москвой,В Москве ограничат движение в связи с Днем Победы,-1,"[люблян, отпраздновать, день, победа, вместе, ...","[москва, ограничить, движение, связь, днём, по..."
7225,Девять самолетов ВВС разгонят облака над Москв...,В Москве ограничат движение в связи с Днем Победы,-1,"[девять, самолёт, ввс, разогнать, облако, моск...","[москва, ограничить, движение, связь, днём, по..."


In [0]:
data['text_1_norm'] = data['text_1'].apply(normalize)
data['text_2_norm'] = data['text_2'].apply(normalize)

In [0]:
def left_context(words, i, window):
    if i < window:
        return [words[j] for j in range(i)]
    else:
       return [words[j] for j in range(i-window, i)]

def right_context(words, i, window):
    n = len(words)
    if i > n-window:
        return [words[j] for j in range(i+1, n)]
    else:
       return [words[j] for j in range(i+1, i+window)]

def get_words_in_context(words, window=3):
    words_in_context = [[words[i], left_context(words, i, window)+right_context(words, i, window)] for i in range(len(words))]
    return words_in_context

In [0]:
words = [0,1,2,3,4,5,6,7,8,9]
get_words_in_context(words)

[[0, [1, 2]],
 [1, [0, 2, 3]],
 [2, [0, 1, 3, 4]],
 [3, [0, 1, 2, 4, 5]],
 [4, [1, 2, 3, 5, 6]],
 [5, [2, 3, 4, 6, 7]],
 [6, [3, 4, 5, 7, 8]],
 [7, [4, 5, 6, 8, 9]],
 [8, [5, 6, 7, 9]],
 [9, [6, 7, 8]]]

In [9]:
!wget 'https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib'
vm = adagram.VectorModel.load('all.a010.p10.d300.w5.m100.nonorm.slim.joblib')

--2020-03-05 20:09:44--  https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.110.157
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.110.157|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1462416741 (1.4G) [application/x-www-form-urlencoded]
Saving to: ‘all.a010.p10.d300.w5.m100.nonorm.slim.joblib.1’

all.a010.p10.d300.w 100%[===================>]   1.36G  54.4MB/s    in 25s     

2020-03-05 20:10:09 (55.2 MB/s) - ‘all.a010.p10.d300.w5.m100.nonorm.slim.joblib.1’ saved [1462416741/1462416741]



In [0]:
def get_embedding_adagram(text, vm=vm, window=3, dim=300):
    
    word2context = get_words_in_context(text, window)
    
    
    vectors = np.zeros((len(word2context), dim))
    
    for i, (word, context) in enumerate(word2context):
        
        try:
            chosen_sence = vm.disambiguate(word, context).argmax()
            v = vm.sense_vector(word, chosen_sence)
            vectors[i] = v
        
        except (KeyError, ValueError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector

In [0]:
import sklearn
data['vector_1'] = data['text_1_norm'].apply(get_embedding_adagram)
data['vector_2'] = data['text_2_norm'].apply(get_embedding_adagram)

In [0]:
import scipy
cos = []
for index, row in data.iterrows():
    cos.append(scipy.spatial.distance.cosine(row['vector_1'], row['vector_2']))
data['cos'] = cos

In [0]:
def chebyshev(vector1, vector2, n=300):
  return max([abs(vector1[i]-vector2[i]) for i in range(n)])

In [0]:
cheb = []
for index, row in data.iterrows():
    cheb.append(chebyshev(list(row['vector_1']), list(row['vector_2'])))
data['chebyshev'] = cheb

Прежде чем использовать расстояния, попробуем посмотреть качество работы на самих векторах

In [0]:
vectors = []
for index, row in data.iterrows():
    vectors.append(list(row['vector_1'])+list(row['vector_2']))
X = pd.DataFrame(vectors)

In [0]:
y = data['label']

In [0]:
model1 = sklearn.linear_model.LogisticRegression()
model2 = sklearn.linear_model.LogisticRegression(C=0.5)
model3 = sklearn.linear_model.LogisticRegression(C=2)

In [0]:
from sklearn.metrics import f1_score
f1 = sklearn.metrics.make_scorer(f1_score, average='macro')

In [0]:
from statistics import mean
mean(sklearn.model_selection.cross_val_score(model1, X, y, scoring=f1))

0.3850605245282466

In [0]:
mean(sklearn.model_selection.cross_val_score(model2, X, y, scoring=f1))

0.3856492709756064

In [0]:
mean(sklearn.model_selection.cross_val_score(model3, X, y, scoring=f1))

0.38131395248781197

По векторам результат не сильно лучше случайного угадывания
Попробуем по расстоянию - косинусному и Чебышева (посчитаны выше)

In [0]:
X = data[['cos', 'chebyshev']]

In [0]:
X[np.isnan(X)] = 0

In [0]:
mean(sklearn.model_selection.cross_val_score(model1, X, y, scoring=f1))

0.4611313654455839

In [0]:
mean(sklearn.model_selection.cross_val_score(model2, X, y, scoring=f1))

0.4467506328753564

In [0]:
mean(sklearn.model_selection.cross_val_score(model3, X, y, scoring=f1))

0.4689463627833923

Лучше всего работает третья модель, натренированная на расстояниях между векторами.
Посмотрим, есть ли случаи, когда она предсказывает -1 там, где полагается 1, или наоборот

In [0]:
train_data = data[:5000]
test_data = data[5000:]

In [0]:
X = train_data[['cos', 'chebyshev']]
X[np.isnan(X)] = 0
y = train_data['label']

In [31]:
model3.fit(X,y)

LogisticRegression(C=2, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
X_test = test_data[['cos', 'chebyshev']]
X_test[np.isnan(X_test)]=0

In [0]:
predicted_label = model3.predict(X_test)

In [0]:
data_to_show = test_data[['text_1', 'text_2', 'label']]
data_to_show['predicted_label'] = predicted_label

In [43]:
mistakes=pd.DataFrame({'text_1':[], 'text_2':[], 'label':[], 'predicted_label':[]})
for index, row in data_to_show.iterrows():
    if (row['label']==1 and row['predicted_label']==-1) or  (row['label']==-1 and row['predicted_label']==1):
        mistakes.iloc[len(mistakes)] = row
mistakes

,text_1,text_2,label,predicted_label


Грубых ошибок нет

Задание 2

In [0]:
def lesk( word, sentence ):
    if type(sentence) is str:
        tokens = [token.strip() for token in sentence.split()]
    elif type(sentence) is list:
        tokens = sentence
    bestsense = 0
    maxoverlap = 0
    synsets = [wn.synsets(word)[i].definition() for i in range(len(wn.synsets(word)))]
    
    for i, syns in enumerate(synsets):
        n=0
        syns_tokens = syns.split()
        for token in tokens:
            if token in syns_tokens:
                n +=1
        if n > maxoverlap:
            bestsense = i
            maxoverlap = n
    return bestsense

In [0]:
import nltk
nltk.download('wordnet')
wn.synsets('day')[1].definition()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


'some point or period in time'

In [0]:
uploaded = files.upload()

Saving corpus_wsd_50k.txt to corpus_wsd_50k.txt


In [0]:
corpus_wsd = []
corpus = open('corpus_wsd_50k.txt').read().split('\n\n')
for sent in corpus:
    corpus_wsd.append([s.split('\t') for s in sent.split('\n')])

Посмотрим, какое будет качество, если везде предсказывать нулевое значение

In [0]:
def baseline_lesc(corpus):
  correct=0
  wrong=0
  for sentence in corpus:
        context = [word[1] for word in sentence if len(word) == 3]
        for word in sentence:
            if word[0] and len(word) == 3:
                lemma = word[1]
                if wn.synsets(lemma)[0] == wn.lemma_from_key(word[0]).synset():
                    correct +=1
                else:
                    wrong += 1 
  accuracy = correct/(correct+wrong)
  return accuracy

print(baseline_lesc(corpus_wsd[:1000]))

0.5325184501845018


In [0]:
def check_lesk(corpus):
    correct = 0
    wrong = 0
    for sentence in corpus:
        context = [word[1] for word in sentence if len(word) == 3]
        for word in sentence:
            if word[0] and len(word) == 3: #проверять предсказание для однозначных слов неинтересно
                lemma = word[1]
                if wn.synsets(lemma)[lesk(lemma, context)] == wn.lemma_from_key(word[0]).synset():
                    correct +=1
                else:
                    wrong += 1
    accuracy = correct/(correct+wrong)
    return accuracy


In [0]:
check_lesk(corpus_wsd[:1000])

0.35389760147601473

Улучшение алгоритма Леска:
Будем делать стеминг и игнорировать стопслова

In [0]:
stopwords = stopwords.words('english')

In [0]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [0]:
def lesk(word, sentence):
    if type(sentence) is str:
        tokens = [token.strip() for token in sentence.split()]
    elif type(sentence) is list:
        tokens = sentence
    tokens = [ps.stem(token) for token in tokens if token not in stopwords]
    bestsense = 0
    maxoverlap = 0
    synsets = [wn.synsets(word)[i].definition() for i in range(len(wn.synsets(word)))]
    
    for i, syns in enumerate(synsets):
        n=0
        syns_tokens = syns.split()
        syns_tokens = [ps.stem(token) for token in syns_tokens if token not in stopwords]
        for token in tokens:
            if token in syns_tokens:
                n +=1
        if n > maxoverlap:
            bestsense = i
            maxoverlap = n
    return bestsense

In [0]:
check_lesk(corpus_wsd[:1000])

0.4143219557195572

Стеминг улучшил показатель, но по-прежнему не превзошел бейзлайн - выбор нулевого значения